### Compute and compare mass-size relation b/w XLSSC 122 and 3DHST
- Dec 2/2020

### Load modules

In [2]:
from __future__ import division
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules')
from GalfitPyWrap import galfitwrap as gf
import galfit_helpers as gfh
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import glob
from scipy import ndimage
from astropy import units as u
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy.io import fits, ascii
from astropy.table import Table, hstack, join
import pickle
import fnmatch
import multiprocessing as mp
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DHST_df = pd.read_csv('/data/emiln/XLSSC122_GalPops/Analysis/Results/3dhst_galfit_full.csv')
XLSSC122_df = pd.read_csv('/data/emiln/XLSSC122_GalPops/Analysis/Results/hst_galfit_full.csv')